In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

% matplotlib inline

In [2]:
data = pd.read_csv("data/DOHMH_unique_inspections.csv")

In [3]:
data.columns.values

array(['CAMIS', 'INSPECTION DATE', 'DBA', 'BORO', 'BUILDING', 'STREET',
       'ZIPCODE', 'PHONE', 'CUISINE DESCRIPTION', 'SCORE', 'GRADE',
       'GRADE DATE', 'RECORD DATE', 'INSPECTION TYPE'], dtype=object)

In [4]:
data["INSPECTION DATE"] = pd.to_datetime(data["INSPECTION DATE"], infer_datetime_format=True)
data["GRADE DATE"] = pd.to_datetime(data["GRADE DATE"], infer_datetime_format=True)
data["RECORD DATE"] = pd.to_datetime(data["RECORD DATE"], infer_datetime_format=True)

In [5]:
# newestInspect = lambda x: x.loc[x["INSPECTION DATE"].argmax()]
data["NEWEST DATE"] = data.groupby("CAMIS")["INSPECTION DATE"].transform(max)

In [6]:
data = data[data["INSPECTION DATE"]==data["NEWEST DATE"]]

In [7]:
coffeeData = data[(data["CUISINE DESCRIPTION"]=="CafÃ©/Coffee/Tea")|
                  (data["CUISINE DESCRIPTION"]=="Donuts")].copy()

In [8]:
coffeeData["storetype"] = "other"
coffeeData.loc[coffeeData["DBA"].str.contains("dunkin.*donuts", case=False), "storetype"] = "dunkin"
coffeeData.loc[coffeeData["DBA"].str.contains("starbuck", case=False), "storetype"] = "starbucks"

In [9]:
coffeeData = coffeeData.reset_index(drop=True)
#coffeeData = coffeeData.drop("level_0", axis=1)


coffeeData = pd.concat([coffeeData, pd.get_dummies(coffeeData["storetype"])], axis=1)
#coffeeData.index.levels()

In [10]:
coffeeData.columns

Index([u'CAMIS', u'INSPECTION DATE', u'DBA', u'BORO', u'BUILDING', u'STREET',
       u'ZIPCODE', u'PHONE', u'CUISINE DESCRIPTION', u'SCORE', u'GRADE',
       u'GRADE DATE', u'RECORD DATE', u'INSPECTION TYPE', u'NEWEST DATE',
       u'storetype', u'dunkin', u'other', u'starbucks'],
      dtype='object')

In [11]:
zipGroups = coffeeData.groupby("ZIPCODE")[["dunkin", "other", "starbucks"]].agg(np.sum)

In [12]:
zipGroups["total"] = zipGroups.sum(axis=1)

In [13]:
for brand in ["dunkin", "starbucks", "other"]:
    zipGroups[brand + "_prop"] = zipGroups[brand]/zipGroups["total"]

In [15]:
zipGroups["total_prop"] = 1.0

In [16]:
zipGroups = zipGroups.reset_index().rename(index=str, columns={"ZIPCODE": "zip"})

In [17]:
zipGroups.head()

,zip,dunkin,other,starbucks,total,dunkin_prop,starbucks_prop,other_prop,total_prop
0,10001,11.0,29.0,16.0,56.0,0.196429,0.285714,0.517857,1.0
1,10002,3.0,36.0,1.0,40.0,0.075000,0.025000,0.900000,1.0
2,10003,6.0,53.0,9.0,68.0,0.088235,0.132353,0.779412,1.0
3,10004,2.0,12.0,5.0,19.0,0.105263,0.263158,0.631579,1.0
4,10005,1.0,4.0,6.0,11.0,0.090909,0.545455,0.363636,1.0


In [19]:
import json
with open('data/nyc-zip-code.json') as json_file:
    zipshapes = json.load(json_file)

In [20]:
len(zipshapes["features"])

262

In [21]:
zipshapes = pd.DataFrame(map(lambda x: {"zip": x["properties"]["ZIP"],
                                    "area": x["properties"]["Shape_Area"]}, 
zipshapes["features"])).groupby("zip").agg(np.sum).reset_index()

In [22]:
zipshapes["zip"] = zipshapes["zip"].astype(int)

In [23]:
zipshapes.dtypes

zip       int64
area    float64
dtype: object

In [24]:
zipGroups = zipGroups.merge(zipshapes, on="zip", how="left")

In [25]:
zipGroups['area'] = zipGroups['area'] / 27878400.0

In [26]:
zipGroups

,zip,dunkin,other,starbucks,total,dunkin_prop,starbucks_prop,other_prop,total_prop,area
0,10001,11.0,29.0,16.0,56.0,0.196429,0.285714,0.517857,1.0,0.638306
1,10002,3.0,36.0,1.0,40.0,0.075000,0.025000,0.900000,1.0,0.942670
2,10003,6.0,53.0,9.0,68.0,0.088235,0.132353,0.779412,1.0,0.557363
3,10004,2.0,12.0,5.0,19.0,0.105263,0.263158,0.631579,1.0,0.486212
4,10005,1.0,4.0,6.0,11.0,0.090909,0.545455,0.363636,1.0,0.074714
5,10006,1.0,0.0,3.0,4.0,0.250000,0.750000,0.000000,1.0,0.061576
6,10007,5.0,7.0,4.0,16.0,0.312500,0.250000,0.437500,1.0,0.191138
7,10009,3.0,20.0,1.0,24.0,0.125000,0.041667,0.833333,1.0,0.570460
8,10010,4.0,18.0,8.0,30.0,0.133333,0.266667,0.600000,1.0,0.350393
9,10011,4.0,37.0,8.0,49.0,0.081633,0.163265,0.755102,1.0,0.649909


In [28]:
zipGroups.to_csv("data/coffee_per_zip.csv", float_format='%.2f')